# PyCitySchools Analysis

In [1]:
# Dependencies and Setup
import pandas as pd
import os

# File to Load (Remember to Change These)
school_data_to_load = os.path.join('Resources', 'schools_complete.csv')
student_data_to_load = os.path.join('Resources', 'students_complete.csv')

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


### District Summary

In [2]:
# Calculate the total number of schools
total_schools = school_data_complete['school_name'].nunique()

# Calculate the total number of students
total_students = school_data_complete['Student ID'].nunique()

# Calculate the total budget
total_budget = school_data['budget'].sum()

# Calculate the average math score
math_score_avg = school_data_complete['math_score'].mean()

# Calculate the average reading score
reading_score_avg = school_data_complete['reading_score'].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
passing_math_pct = (((student_data['math_score'] >= 70).sum())/total_students)*100

# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_pct = (((student_data['reading_score'] >= 70).sum())/total_students)*100


# Calculate the percentage of students who passed math and reading (% Overall Passing)
overall_passing = ((((student_data['reading_score'] >= 70) & (student_data['math_score'] >= 70)).sum())/total_students)*100

# Create a dataframe to hold the above results
district_summary_header = ['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 
           'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing Rate']

district_summary_values = [total_schools, total_students, total_budget, math_score_avg, reading_score_avg, 
          passing_math_pct, passing_reading_pct, overall_passing]


district_summary_df = pd.DataFrame([district_summary_values], columns=district_summary_header)

# Formatting
district_summary_df = district_summary_df.round(2)
district_summary_df['Total Budget'] = district_summary_df['Total Budget'].map("${:,.0f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,"$24,649,428",78.99,81.88,74.98,85.81,65.17


### School Summary

In [3]:
# Determine the school name and type
school_type = school_data.set_index(['school_name'])['type']

# Determine total students per school
per_school_students = school_data_complete['school_name'].value_counts()

# Determine total school budget
per_school_budget = school_data_complete.groupby(['school_name']).mean()['budget']

# Determine per student budget
per_student_budget = per_school_budget/per_school_students

# Calculate average math score
per_school_math_score = school_data_complete.groupby(['school_name']).mean()['math_score']

# Calculate average reading score
per_school_reading_score = school_data_complete.groupby(['school_name']).mean()['reading_score']

# Calculate % passing math
per_school_passing_math_pct = (school_data_complete[school_data_complete['math_score'] >=70].groupby(['school_name'])['student_name'].count()/per_school_students)*100

# Calculate % passing reading
per_school_passing_reading_pct = (school_data_complete[school_data_complete['reading_score'] >=70].groupby(['school_name'])['student_name'].count()/per_school_students)*100

# Calculate % overall passing (the percentage of student that passed math and reading)
per_school_overall_passing = (school_data_complete[(school_data_complete['math_score'] >=70)&(school_data_complete['reading_score'] >=70)].groupby(['school_name'])['student_name'].count()/per_school_students)*100

# Create a dataframe for school summary

school_summary_df = pd.DataFrame({'School Type': school_type,
                                  'Total Students': per_school_students,
                                  'Total School Budget': per_school_budget,
                                  'Per Student Budget': per_student_budget,
                                  'Average Math Score': per_school_math_score,
                                  'Average Reading Score': per_school_reading_score,
                                  '% Passing Math': per_school_passing_math_pct,
                                  '% Passing Reading': per_school_passing_reading_pct,
                                  '% Overall Passing Rate': per_school_overall_passing})
# Formatting
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map("${:,.2f}".format)
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map("${:,.2f}".format)
school_summary_df = school_summary_df.round(2)

school_summary_df


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


### Top Performing Schools (By % Overall Passing)

In [4]:
top_perf_schools = school_summary_df.sort_values(['% Overall Passing Rate'], ascending=False)
top_perf_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13,97.04,91.33
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27,97.31,90.95
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39,97.14,90.60
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87,96.54,90.58
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59,95.95,90.54


### Bottom Performing Schools (By % Overall Passing)

In [5]:
top_perf_schools = school_summary_df.sort_values(['% Overall Passing Rate'], ascending=True)
top_perf_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37,80.22,52.99
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99,80.74,53.20
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68,81.32,53.51
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75,80.86,53.53
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06,81.22,53.54


### Math Scores by Grade

In [6]:
# Create a pandas series for each grade using a conditional statement
ninth_grade = school_data_complete[(school_data_complete['grade'] == '9th')]
tenth_grade = school_data_complete[(school_data_complete['grade'] == '10th')]
eleventh_grade = school_data_complete[(school_data_complete['grade'] == '11th')]
twelfth_grade = school_data_complete[(school_data_complete['grade'] == '12th')]

# Group each series by school
ninth_grade_math = ninth_grade.groupby(['school_name']).mean()['math_score']
tenth_grade_math = tenth_grade.groupby(['school_name']).mean()['math_score']
eleventh_grade_math = eleventh_grade.groupby(['school_name']).mean()['math_score']
twelfth_grade_math = twelfth_grade.groupby(['school_name']).mean()['math_score']

# Combine the series into a dataframe
math_scores_per_grade_df = pd.DataFrame({'9th': ninth_grade_math, 
                                         '10th': tenth_grade_math,
                                         '11th': eleventh_grade_math, 
                                         '12th': twelfth_grade_math})

# Formatting
math_scores_per_grade_df = math_scores_per_grade_df.round(2)
math_scores_per_grade_df.index.name = None

math_scores_per_grade_df

,9th,10th,11th,12th
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86
Pena High School,83.63,83.37,84.33,84.12


### Reading Score by Grade 

In [7]:
# Group each series by school
ninth_grade_reading = ninth_grade.groupby(['school_name']).mean()['reading_score']
tenth_grade_reading = tenth_grade.groupby(['school_name']).mean()['reading_score']
eleventh_grade_reading = eleventh_grade.groupby(['school_name']).mean()['reading_score']
twelfth_grade_reading = twelfth_grade.groupby(['school_name']).mean()['reading_score']

# Combine the series into a dataframe
reading_scores_per_grade_df = pd.DataFrame({'9th': ninth_grade_reading, 
                                         '10th': tenth_grade_reading,
                                         '11th': eleventh_grade_reading, 
                                         '12th': twelfth_grade_reading})

# Formatting
reading_scores_per_grade_df = reading_scores_per_grade_df.round(2)
reading_scores_per_grade_df.index.name = None

reading_scores_per_grade_df

,9th,10th,11th,12th
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23
Pena High School,83.81,83.61,84.34,84.59


### Scores by School Spending

In [9]:
# Establish the bins
bins = [0, 585, 630, 645, 680]
bins_group_names = ['<$585','$585-615', '$615-645', '$645-680']

# Calculate breaks down school performances based on average Spending Ranges
school_summary_df['Spending Ranges (Per Student)'] = pd.cut(per_student_budget, bins, labels=bins_group_names)

spending_avg_math_score = school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['Average Math Score']
spending_avg_reading_score = school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['Average Reading Score']
spending_pct_passing_math = school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Math']
spending_pct_passing_reading = school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Passing Reading']
spending_overall_passing_rate = school_summary_df.groupby(['Spending Ranges (Per Student)']).mean()['% Overall Passing Rate']

# Create a dataframe for scores by school spending
scores_school_spending_df = pd.DataFrame({'Average Math Score' : spending_avg_math_score,
                                         'Average Reading Score': spending_avg_reading_score,
                                         '% Passing Math': spending_pct_passing_math,
                                         '% Passing Reading': spending_pct_passing_reading,
                                         '% Overall Passing Rate': spending_overall_passing_rate})

# Formatting
scores_school_spending_df = scores_school_spending_df.round(2)

scores_school_spending_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.45,83.94,93.46,96.61,90.37
$585-615,81.90,83.16,87.13,92.72,81.42
$615-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


### Scores by School Size

In [11]:
# Establish the bins
bins = [0, 1000, 2000, 5000]
bins_group_names = ['Small (<1000)','Medium (1000-2000)', 'Large (2000-5000)']

# Calculate breaks down school performances based on school size
school_summary_df['School Size'] = pd.cut(school_summary_df['Total Students'], bins, labels=bins_group_names)

school_size_avg_math_score = school_summary_df.groupby(['School Size']).mean()['Average Math Score']
school_size_avg_reading_score = school_summary_df.groupby(['School Size']).mean()['Average Reading Score']
school_size_pct_passing_math = school_summary_df.groupby(['School Size']).mean()['% Passing Math']
school_size_pct_passing_reading = school_summary_df.groupby(['School Size']).mean()['% Passing Reading']
school_size_overall_passing_rate = school_summary_df.groupby(['School Size']).mean()['% Overall Passing Rate']

# Create a dataframe for scores by school size
scores_school_size_df = pd.DataFrame({'Average Math Score' : school_size_avg_math_score,
                                         'Average Reading Score': school_size_avg_reading_score,
                                         '% Passing Math': school_size_pct_passing_math,
                                         '% Passing Reading': school_size_pct_passing_reading,
                                         '% Overall Passing Rate': school_size_overall_passing_rate})

# Formatting
scores_school_size_df = scores_school_size_df.round(2)

scores_school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.87,93.60,96.79,90.62
Large (2000-5000),77.74,81.34,69.96,82.77,58.28


### Scores by School Type

In [12]:
# Calculate breaks down school performances based on school type
school_type_avg_math_score = school_summary_df.groupby(['School Type']).mean()['Average Math Score']
school_type_avg_reading_score = school_summary_df.groupby(['School Type']).mean()['Average Reading Score']
school_type_pct_passing_math = school_summary_df.groupby(['School Type']).mean()['% Passing Math']
school_type_pct_passing_reading = school_summary_df.groupby(['School Type']).mean()['% Passing Reading']
school_type_overall_passing_rate = school_summary_df.groupby(['School Type']).mean()['% Overall Passing Rate']

# Create a dataframe for scores by school type
scores_school_type_df = pd.DataFrame({'Average Math Score': school_type_avg_math_score,
                                         'Average Reading Score': school_type_avg_reading_score,
                                         '% Passing Math': school_type_pct_passing_math,
                                         '% Passing Reading': school_type_pct_passing_reading,
                                         '% Overall Passing Rate': school_type_overall_passing_rate})

# Formatting
scores_school_type_df = scores_school_type_df.round(2)

scores_school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
